# Getting Started with RedisVL

RedisVL is a Python library with a dedicated CLI to help load and create vector search indices within Redis. While 

This notebook will walk through
1. Preparing a dataset with vectors.
2. Writing data schema for ``redis``
3. Loading the data and creating a vector search index
4. Performing queries

Before running this notebook, be sure to
1. Have installed ``redisvl`` and have that environment active for this notebook.
2. Have a running Redis instance with RediSearch > 2.4 running.

## Data Preparation

For this example, we will use the following overly simplified dataset


In [1]:
import numpy as np
from pprint import pprint

data = [
    {'user': 'john', 'age': 1, 'job': 'engineer', 'credit_score': 'high'},
    {'user': 'mary', 'age': 2, 'job': 'doctor', 'credit_score': 'low'},
    {'user': 'joe', 'age': 3, 'job': 'dentist', 'credit_score': 'medium'}
]

This will make up 3 entries in Redis (hashes) each with 4 sub-keys (users, age, job, credit_score).

Now, we want to add vectors to represent each user. These are just dummy vectors to illustrate the point, but more complex vectors can be created and used as well. For more information on creating embeddings, see this [article](https://mlops.community/vector-similarity-search-from-basics-to-production/).


In [2]:
# converted to bytes for redis
vectors = [
    np.array([0.1, 0.1, 0.5], dtype=np.float32).tobytes(),
    np.array([0.1, 0.1, 0.5], dtype=np.float32).tobytes(),
    np.array([0.9, 0.9, 0.1], dtype=np.float32).tobytes(),
]

for record, vector in zip(data, vectors):
    record["user_embedding"] = vector

pprint(data)

[{'age': 1,
  'credit_score': 'high',
  'job': 'engineer',
  'user': 'john',
  'user_embedding': b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'},
 {'age': 2,
  'credit_score': 'low',
  'job': 'doctor',
  'user': 'mary',
  'user_embedding': b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'},
 {'age': 3,
  'credit_score': 'medium',
  'job': 'dentist',
  'user': 'joe',
  'user_embedding': b'fff?fff?\xcd\xcc\xcc='}]


As seen above, the vectors themselves need to be turned into bytes before they can be loaded into Redis. Using ``NumPy``, this is fairly trivial. 

Our dataset is now ready to be used with ``redisvl``

## Define Index Schema

In order for ``redisvl`` to be flexible for many types of data, it uses a schema specified in either a python dictionary or a yaml file. There are a couple main components

1. index specification
2. field specification

The index specification determines how data will be stored in Redis. This includes
- ``name``: the name of the index
- (Optional) ``prefix``: key prefix for each loaded entry
- (Optional) ``key_field``: field within the dataset to use as unique keys. If not specified, a unique key will be generated for each entry.

The field specification determines what fields within the dataset will be available for queries. Each field corresponds to the name of a **column** within the dataset. The values within each specified column are arguments for the creation of that index that correspond directly to ``redis-py`` arguments.

So for example, given the above dataset, the following schema can be used.


```yaml

index:
  name: user_index
  prefix: v1
  key_field: user
  storage_type: hash

fields:
    # define tag fields
    tag:
        - name: user
        - name: job
        - name: credit_store
    # define numeric fields
    numeric:
        - name: age
    # define vector fields
    vector:
        - name: user_embedding
        algorithm: hnsw
        distance_metric: cosine


```

In Python, this looks like:

In [3]:
schema = {
    "index": {
        "name": "user_index",
        "prefix": "v1",
        "key_field": "user",
        "storage_type": "hash",
    },
    "fields": {
        "tag": [{"name": "credit_score"}],
        "text": [{"name": "job"}],
        "numeric": [{"name": "age"}],
        "vector": [{
                "name": "user_embedding",
                "dims": 3,
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"}
        ]
    },
}


## Create a ``SearchIndex``

With the data and the index schema defined, we can now use ``redisvl`` as a library to create a search index as follows.

Note that at this point, the index will have no entries. With Redis, this is fine as new entries from this index (or that follow the schema) will automatically be indexed in the background in Redis.

In [4]:
from redisvl.index import SearchIndex

# construct a search index from the schema
index = SearchIndex.from_dict(schema) # or SearchIndex.from_yaml("schema.yaml") for yaml files

# connect to local redis instance
index.connect("redis://localhost:6379")

# create the index (no data yet)
index.create(overwrite=True)

In [6]:
# use the CLI to see the created index
!rvl index listall

16:42:28 sam.partee-NW9MQX5Y74 redisvl.cli.index[4009] INFO Indices:
16:42:28 sam.partee-NW9MQX5Y74 redisvl.cli.index[4009] INFO 1. user_index


## Load Data into the Index

Now that an index exists, data can be loaded into redis through the ``SearchIndex.load()`` function

In [7]:
# load expects an iterable of dictionaries
index.load(data)

## Executing Queries

Next we will run a vector query on our newly populated index. This example will use a simple vector to demonstrate how vector similarity works. Vectors in production will be much larger than 3 floats and often require Machine Learning models (i.e. Huggingface sentence transformers) or an embeddings API (Cohere, OpenAI) to create.

In [8]:
from redisvl.query import VectorQuery

# create a vector query returning a number of results
# with specific fields to return.
query = VectorQuery(
    vector=[0.1, 0.1, 0.5],
    vector_field_name="user_embedding",
    return_fields=["user", "age", "job", "credit_score", "vector_distance"],
    num_results=3
)

# use the SearchIndex instance (or Redis client) to execute the query
results = index.query(query)

In [10]:
for doc in results.docs:
    print("Score:", doc.vector_distance)
    print(doc)


Score: 0
Document {'id': 'v1:john', 'payload': None, 'vector_distance': '0', 'user': 'john', 'age': '1', 'job': 'engineer', 'credit_score': 'high'}
Score: 0
Document {'id': 'v1:mary', 'payload': None, 'vector_distance': '0', 'user': 'mary', 'age': '2', 'job': 'doctor', 'credit_score': 'low'}
Score: 0.653301358223
Document {'id': 'v1:joe', 'payload': None, 'vector_distance': '0.653301358223', 'user': 'joe', 'age': '3', 'job': 'dentist', 'credit_score': 'medium'}


## Connecting to an Existing Index

If you have an existing index, you can connect to it using the ``SearchIndex.from_existing()`` function. This will return a ``SearchIndex`` object that can be used to execute queries.


In [11]:
from redis import Redis

# initialize a redis client to connect to existing index
redis_client = Redis("localhost", 6379)

# create a new SearchIndex instance from an existing index
existing_index = SearchIndex.from_existing(redis_client, "user_index")

# run the same query
existing_index.query(query)

for doc in results.docs:
    print("Score:", doc.vector_distance)
    print(doc)

ResponseError: Unknown Index name

## Asynchronous Search

The AsyncSearchIndex class allows for queries, index creation, and data loading to be done asynchronously. This is useful for large datasets that may take a long time to load into Redis, for queries that may take a long time to execute, or for asynchronous applications that need to execute queries in the background like a FastAPI application.

In [12]:
## Asynchronous Search

from redisvl.index import AsyncSearchIndex

# construct a search index from the schema
index = AsyncSearchIndex.from_dict(schema)

# connect to local redis instance
index.connect("redis://localhost:6379")

# create the index (no data yet)
await index.create(overwrite=True)

# load the data again but asynchronously
await index.load(data)

# run the same vector query but asynchronously
results = await index.query(query)

for doc in results.docs:
    print("Score:", doc.vector_distance)
    print(doc)

Score: 0
Document {'id': 'v1:john', 'payload': None, 'vector_distance': '0', 'user': 'john', 'age': '1', 'job': 'engineer', 'credit_score': 'high'}
Score: 0
Document {'id': 'v1:mary', 'payload': None, 'vector_distance': '0', 'user': 'mary', 'age': '2', 'job': 'doctor', 'credit_score': 'low'}
Score: 0.653301358223
Document {'id': 'v1:joe', 'payload': None, 'vector_distance': '0.653301358223', 'user': 'joe', 'age': '3', 'job': 'dentist', 'credit_score': 'medium'}
